In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'e:\\MLs\\MLOPS Project\\SimpleMLopsProject'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 scherma_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(scherma_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_model_train_config(self) -> ModelTrainConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        model_train_config = ModelTrainConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        
        return model_train_config